In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/shopee-code-league-20/_DA_Marketing_Analytics'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_user = pd.read_csv('/kaggle/input/shopee-code-league-20/_DA_Marketing_Analytics/users.csv')

In [ ]:
train = pd.read_csv('/kaggle/input/shopee-code-league-20/_DA_Marketing_Analytics/train.csv')

In [ ]:
test = pd.read_csv('/kaggle/input/shopee-code-league-20/_DA_Marketing_Analytics/test.csv')

In [ ]:
sample = pd.read_csv('/kaggle/input/shopee-code-league-20/_DA_Marketing_Analytics/sample_submission_0_1.csv')

In [ ]:
sample.describe

In [ ]:
df_user.loc[df_user.user_id == 130]

In [ ]:
test.head()

In [ ]:
train.set_index('row_id')

In [ ]:
train_y = train.set_index('row_id').open_flag

In [ ]:
train_y.head()

In [ ]:
train_x = train.set_index('user_id').join(df_user.set_index('user_id')).reset_index()
test_x = test.set_index('user_id').join(df_user.set_index('user_id')).reset_index()

In [ ]:
#train_x.loc[~train_x['last_checkout_day'].str.isdigit(), 'last_checkout_day'].tolist()
train_x.loc[~train_x['last_checkout_day'].str.isdigit(), 'last_checkout_day'] = 0
test_x.loc[~test_x['last_checkout_day'].str.isdigit(), 'last_checkout_day'] = 0

In [ ]:
train_x.head()

In [ ]:
train_x=train_x.drop(['grass_date','user_id','open_flag'],axis=1)
test_x=test_x.drop(['grass_date','user_id'],axis=1)
#train_x=train_x.drop('open_flag',axis=1)

In [ ]:
train_x.loc[~train_x['last_open_day'].str.isdigit(), 'last_open_day']=0
test_x.loc[~test_x['last_open_day'].str.isdigit(), 'last_open_day']=0

In [ ]:
train_x.loc[~train_x['last_login_day'].str.isdigit(), 'last_login_day'] = 0
test_x.loc[~test_x['last_login_day'].str.isdigit(), 'last_login_day'] = 0

In [ ]:
train_x.dtypes

In [ ]:
train_x.last_checkout_day = train_x.last_checkout_day.astype('int32')
train_x.last_open_day  = train_x.last_open_day.astype('int32')
train_x.last_login_day  = train_x.last_login_day.astype('int32')
test_x.last_checkout_day = test_x.last_checkout_day.astype('int32')
test_x.last_open_day  = test_x.last_open_day.astype('int32')
test_x.last_login_day  = test_x.last_login_day.astype('int32')

In [ ]:
train_x = train_x.set_index('row_id')
test_x = test_x.set_index('row_id')

In [ ]:
train_x = train_x.sort_index()
test_x = test_x.sort_index()

In [ ]:
train_x.head()

In [ ]:
train_x.describe()

In [ ]:
train_y.reset_index()

In [ ]:
test_x.head()

In [ ]:
train_y.head()

In [ ]:
train_y = train_y.reset_index().drop('row_id',axis=1)

In [ ]:
train_y.describe

1. finished manage train_x and train_y

ML


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num = SimpleImputer()
OH = OneHotEncoder()
numerical_cols = ['attr_1','attr_2','attr_3','age']
OH_cols = ['domain']

preprocessor = ColumnTransformer(
                transformers =[
                    ('num',num,numerical_cols),
                    ('OH',OH,OH_cols)
                ])

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 100)
my_pipeline = Pipeline( steps=[
    ('preprocessor',preprocessor),
    ('model',model)
])
my_pipeline.fit(train_x,train_y)

In [ ]:
result = my_pipeline.predict(test_x)

In [ ]:
summit = np.where(result > 0.5, 1, 0)

In [ ]:
submission = pd.DataFrame({'row_id': test_x.index.values,'open_flag':summit})

In [ ]:
submission

In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE:', mean_absolute_error(sample.open_flag, submission.open_flag))